# Hazardous Material Sign Detection

In [ ]:
import torch
from ultralytics import YOLO
import random
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

### Install packages

In [ ]:
!pip install -r requirements.txt

### Dataset

The hazmat labels of the RoboCupRescue League are:
* explosives
* blasting agents
* flammable gas
* non flammable gas
* oxygen
* fuel-oil
* dangerous-when-wet
* flammable-solid
* spontaneously-combistible
* oxidizer
* organic-peroxide
* inhalation-hazard
* poison
* radioactive
* corrosive

In [ ]:
!unzip -n ./dataset/hazmat-placard-image-dataset -d dataset

Get path

In [ ]:
# get absolute path
path = !pwd
path = path[0]

### Structure of the directory
```
.
└── hazmat_detection/
    ├── checkpoints/
    │   └── ...
    ├── dataset/
    │   └── HazMat.v16i.yolov11/
    │       ├── data.yaml
    │       │
    │       ├── train/
    │       │   ├── images/
    │       │   │
    │       │   └── labels/
    │       │  
    │       ├── val/
    │       │
    │       ├── test/
    │       │ 
    │       └── ...
    ├── utils/
    │   └── ...
    ├── model.ipynb
    ├── requirements.txt
    ├── train.py
    └── test.py
```

## Data distribution

Check data distribution in the training, validation and test

In [ ]:
NUM_CLASSES = 15 # number of labels
classes = range(0, NUM_CLASSES)
data = pd.DataFrame()
empty = 0 # count number of empty inputs

# for each dataset
for dataset in ["train","valid","test"]:
    directory = f"dataset/HazMat.v16i.yolov11/{dataset}/labels"
    filenames = os.listdir(directory)
    classCount = {el: 0 for el in range(0,NUM_CLASSES)}
    total = 0
    for filename in filenames:
        with open(os.path.join(directory, filename), 'r') as file:
            lines = file.readlines()
            if len(lines) == 0:
                empty += 1
            else:
                for line in lines:
                    classCount[int(line.strip().split(" ")[0])] += 1
                    total += 1
    # print
    values = pd.array(list(classCount.values()))/total
    data[dataset] = values.copy()

print(f"There are {empty} empty lines")
# plot distribution of labels in training, validation and test set
plt.grid(alpha=0.5)
sns.lineplot(data=data)
plt.xlabel("Class")
plt.ylabel("Frequency")
plt.xticks(range(0,NUM_CLASSES))
plt.show()

## Train model

In [ ]:
IMAGE_SIZE = 640 # size for YOLO model

In [ ]:
if torch.cuda.is_available():
    print("Cuda")
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    print("Cuda not available, use mps")
    device = torch.device("mps")
else:
    print("Use CPU")
    device = torch.device("cpu")

Load model

In [ ]:
# load pretrained model
model = YOLO("./checkpoints/yolo11n.pt")

In [ ]:
results = model.train(data=f"{path}/dataset/HazMat.v16i.yolov11/data.yaml", epochs=20, batch=32, device=device,
        imgsz=IMAGE_SIZE,
        augment=True,
        flipud=0.5,    # Probability for vertical flip
        fliplr=0.5,    # Probability for horizontal flip
        hsv_h=0.1,    # Adjust hue
        hsv_s=0.7,     # Adjust saturation
        hsv_v=0.4,     # Adjust value
        degrees=180, # randomly rotate -180 to +180
        translate=0.2, # translate image horizontally and vertically
        scale=0.1, # scale image to simulate different distance
        shear=0.1, # mimic image seen from different angle
        # perspective=0.1, # perspective transformation
        # cos_lr=True, # cosine learning rate scheduler
    )

In [ ]:
results = model.val(
        data=f"{path}/dataset/HazMat.v16i.yolov1/data.yaml",
        epochs=1,
        batch=32,
        imgsz=IMAGE_SIZE,
        device=device
    )

## Test model

In [ ]:
pathTest = f"{path}/dataset/HazMat.v16i.yolov1/images/test"
filenames = os.listdir(pathTest)
random.shuffle(filenames)

for filename in filenames:
    path = os.path.join(pathTest, filename)
    results = model.predict(path)
    for result in results:
        result.show()

## Save model

While training, YOLO Ultralytics creates a folder called ```runs``` where you can find the best and last model trained. You can download them and use them as starting point for finetuning, further training and so on.